<a href="https://colab.research.google.com/github/nur303/Problem-solve/blob/main/Unit_production_removeing%20main%20file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from collections import defaultdict

def remove_unit_productions(grammar):
    # Step 1: Initialize UNIT dictionary to track unit productions (A → B where B is a non-terminal)
    UNIT = defaultdict(set)
    for A in grammar.keys():
        UNIT[A].add(A)  # Include self as a unit production initially

    # Step 2: Find all unit pairs (transitive closure)
    changed = True
    while changed:
        changed = False
        for A in grammar:
            for prod in grammar[A]:
                # Check if prod is a single non-terminal (unit production)
                if len(prod) == 1 and prod in grammar:
                    if prod not in UNIT[A]:
                        UNIT[A].add(prod)
                        changed = True
                    # Add all non-terminals reachable from prod
                    for C in UNIT[prod]:
                        if C not in UNIT[A]:
                            UNIT[A].add(C)
                            changed = True

    # Step 3: Create new grammar with non-unit productions
    new_grammar = defaultdict(set)
    for A in grammar:
        for B in UNIT[A]:
            for prod in grammar[B]:
                # Include only non-unit productions (not a single non-terminal)
                if not (len(prod) == 1 and prod in grammar):
                    new_grammar[A].add(prod)

    # Convert sets to sorted lists for consistent output
    final_grammar = {A: sorted(list(prods)) for A, prods in new_grammar.items()}
    return final_grammar

def remove_unreachable_productions(grammar, start_symbol):
    # Step 1: Find reachable non-terminals using DFS
    reachable = set()
    def find_reachable(symbol):
        if symbol in reachable:
            return
        reachable.add(symbol)
        if symbol in grammar:
            for prod in grammar[symbol]:
                # Extract non-terminals from production (assuming non-terminals are uppercase)
                for char in prod:
                    if char.isupper() and char in grammar:
                        find_reachable(char)

    find_reachable(start_symbol)

    # Step 2: Create new grammar with only reachable non-terminals
    new_grammar = defaultdict(list)
    for A in grammar:
        if A in reachable:
            new_grammar[A] = grammar[A]

    return new_grammar

# Interactive Input
def get_grammar_input():
    print("Enter your CFG productions (format: NonTerminal -> RHS1,RHS2,...). Type 'done' to finish.")
    grammar = defaultdict(list)
    non_terminals = []
    first_non_terminal = None

    while True:
        line = input("Production: ").strip()
        if line.lower() == 'done':
            if not grammar:
                print("Error: No productions entered.")
                return None, None
            return grammar, first_non_terminal
        if '->' not in line and '→' not in line:
            print("Invalid format. Use 'NonTerminal -> RHS1,RHS2,...' or 'NonTerminal → RHS1,RHS2,...'")
            continue
        separator = '->' if '->' in line else '→'
        left, right = line.split(separator)
        left = left.strip()
        if not left or not all(c.isupper() or c.isspace() for c in left):
            print("Invalid non-terminal. Use uppercase letters (e.g., A, S, AB).")
            continue
        rights = [r.strip() for r in right.split(',') if r.strip()]
        if not rights:
            print("Error: No right-hand side provided.")
            continue
        grammar[left].extend(rights)
        non_terminals.append(left)
        if first_non_terminal is None:
            first_non_terminal = left
        print(f"Added: {left} -> {', '.join(rights)}")

# Main execution
if __name__ == "__main__":
    grammar, start_symbol = get_grammar_input()
    if grammar is None:
        print("Exiting due to invalid grammar.")
    else:
        # Remove unit productions
        grammar_without_unit = remove_unit_productions(grammar)

        # Remove unreachable productions
        final_grammar = remove_unreachable_productions(grammar_without_unit, start_symbol)

        # Display original grammar
        print("\nOriginal Grammar:")
        for k, v in grammar.items():
            print(f"{k} -> {', '.join(v)}")

        # Display grammar without unit and unreachable productions
        print("\nGrammar without Unit and Unreachable Productions:")
        if not final_grammar:
            print("No productions remain after removing unit and unreachable productions.")
        else:
            for k, v in final_grammar.items():
                print(f"{k} -> {', '.join(v)}")

Enter your CFG productions (format: NonTerminal -> RHS1,RHS2,...). Type 'done' to finish.
Production: s->aA,B
Invalid non-terminal. Use uppercase letters (e.g., A, S, AB).
Production: S->aA,B
Added: S -> aA, B
Production: A->ba,bb
Added: A -> ba, bb
Production: B->A,bab
Added: B -> A, bab
Production: done

Original Grammar:
S -> aA, B
A -> ba, bb
B -> A, bab

Grammar without Unit and Unreachable Productions:
S -> aA, ba, bab, bb
A -> ba, bb
